In [8]:
from DataProcessing import DataProcessing

In [9]:
import pandas as pd
import time

In [56]:
data_procesing = DataProcessing("car.data")
data_procesing.read_data()
data_procesing.process()
data = data_procesing.encoded_data

In [57]:
class RuleItem():
    def __init__(self):
        self.conditions: dict = {}
        self.class_: str = ""
        self.support: float = 0.0
        self.confidence: float = 0.0
        self.lift: float = 0.0
        self.is_ruleitem = False

    def set_ruleitem(self, item: dict, data: pd.DataFrame, min_support: float = 0.01, min_confidence: float = 0.1):
        # Declare Instance Variables
        support: float = 0.0
        condition_support: float = 0.0
        confidence: float = 0.0
        # Definite Instance Variables
        conditions = item["conditions"]
        class_ = item["class"]
        data_frame_conditions = None

        for key in conditions:
            if (data_frame_conditions is None):
                data_frame_conditions = data[key] == conditions[key]
            else:
                data_frame_conditions &= data[key] == conditions[key]
        data_frame_conditions_with_class = data_frame_conditions & (
            data["class"] == class_)

        support = data[data_frame_conditions_with_class].shape[0] / \
            data.shape[0]
        condition_support = data[data_frame_conditions].shape[0] / \
            data.shape[0]
        if support >= min_support:
            confidence = support / condition_support
            if confidence >= min_confidence:
                self.conditions = conditions
                self.class_ = class_
                self.support = support
                self.confidence = confidence
                self.is_ruleitem = True
                # self.lift = confidence / (data[data["class"] == class_].shape[0] / data.shape[0])
                del data
                return True
        return False

    def print_ruleitem(self):
        if not self.is_ruleitem:
            print("This is not a ruleitem")
            return
        string = ""
        for key, value in self.conditions.items():
            string += f"{key}={value} "
        string += f"=> class={self.class_} "
        string += f"(support={self.support}, confidence={self.confidence}, lift={self.lift})"
        print(string)


class Car():
    def __init__(self, data: pd.DataFrame, min_support: float = 0.01, min_confidence: float = 0.1):
        self.data = data
        self.label = list(self.data['class'].unique())
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.rlue: list = []  # class: ruleitem

    def generate_candidate(self, rule_temp):

        def join(ruleitem1: RuleItem, ruleitem2: RuleItem):
            if ruleitem1.class_ != ruleitem2.class_:
                return None

            condiset1 = set(ruleitem1.conditions)
            condiset2 = set(ruleitem2.conditions)

            if (condiset1 == condiset2):  # 判斷全部的條件屬性是否相等，不一樣表示有不同的條件屬性，可以繼續合併。
                return None

            intersect = condiset1 & condiset2  # 交集

            for item in intersect:  # 判斷交集是否相等，如果相等表示可以合併，不相等表示相同的條件屬性下部分屬性值不同，不可以合併。
                if ruleitem1.conditions[item] != ruleitem2.conditions[item]:
                    return None

            condiset = condiset1 | condiset2  # 合併兩個條件屬性集合
            new_item = {'conditions': {}, 'class': None}
            for item in condiset:
                if item in condiset1:
                    new_item['conditions'][item] = ruleitem1.conditions[item]
                else:
                    new_item['conditions'][item] = ruleitem2.conditions[item]
            new_item['class'] = ruleitem1.class_
            return new_item

        candidate = []
        for i in range(0, len(rule_temp)):
            for j in range(i+1, len(rule_temp)):
                new_item = join(rule_temp[i], rule_temp[j])
                if new_item != None:
                    rule_item = RuleItem()
                    rule_item.set_ruleitem(
                        new_item, self.data, self.min_support, self.min_confidence)
                    if rule_item.is_ruleitem:
                        if rule_item.confidence >= rule_temp[i].confidence and rule_item.confidence >= rule_temp[j].confidence:
                            candidate.append(rule_item)
        return candidate

    def generate_frequent(self):
        label = self.label
        data = self.data
        min_support = self.min_support
        min_confidence = self.min_confidence
        rule_temp = []
        for i in data.columns[:-1]:
            for j in data[i].unique():
                for k in label:
                    ruleitem = RuleItem()
                    candidate = {'conditions': {i: j}, 'class': k}
                    if (ruleitem.set_ruleitem(candidate, data, min_support, min_confidence)):
                        self.rlue.append(ruleitem)
                        rule_temp.append(ruleitem)
        while rule_temp != []:
            candidate = self.generate_candidate(rule_temp)
            print("candidate:", len(candidate))

            if candidate == []:
                rule_temp = []
                break
            rule_temp = []
            for item in candidate:
                self.rlue.append(item)
                rule_temp.append(item)

In [60]:
MIN_SUPPORT = 0.01
MIN_CONFIDENCE = 0.5
car = Car(data, MIN_SUPPORT, MIN_CONFIDENCE)
car.generate_frequent()

candidate: 51
candidate: 369
candidate: 0


In [ ]:
data = data_procesing.encoded_data
columns = list(data.columns)
columns.append("condition_support")
columns.append("support")
columns.append("confidence")
columns.append("lift")
frequent_items_one = []

MIN_SUPPORT = 0.05
MIN_CONFIDENCE = 0.5
# MIN_LIFT = 1.1


label = list(data['class'].unique())

for i in data.columns[:-1]:
    for j in data[i].unique():
        print(i, j)
        pass
        data_conditions = data[i] == j
        for k in label:
            data_conditions_with_label = (
                data_conditions & (data['class'] == k))
            support = data[data_conditions_with_label].shape[0] / data.shape[0]
            data_conditions_supprot = data[data_conditions].shape[0] / \
                data.shape[0]
            confidence = support / data_conditions_supprot
            if (support >= MIN_SUPPORT and confidence >= MIN_CONFIDENCE):
                frequent_items_one.append({
                    'conditions':
                        {
                            i: j
                        },
                    'class': k,
                    'support': support,
                    'confidence': confidence,
                })
# frequent_items_one

0 1
0 2
0 0
0 4
0 3
1 7
1 6
1 4
1 1
1 2
1 0
1 5
1 8
1 3
2 0
2 1
2 2
2 3
2 4
3 9
3 11
3 1
3 12
3 6
3 15
3 7
3 8
3 5
3 10
3 14
3 4
3 0
3 3
3 13
3 2
4 3
4 2
4 1
4 4
4 0
5 4
5 2
5 0
5 3
5 5
5 1
5 6
6 1
6 4
6 6
6 10
6 8
6 12
6 3
6 14
6 5
6 7
6 13
6 0
6 11
6 2
6 9
7 1
7 0
7 5
7 3
7 4
7 2
8 4
8 2
8 1
8 0
8 3
9 1
9 0
10 0
10 1
10 3
10 2
11 0
11 2
11 1
11 3
11 4
12 1
12 0
12 2
12 4
12 3
13 39
13 5
13 23
13 19
13 0
13 26
13 35
13 33
13 16
13 9
13 2
13 11
13 20
13 30
13 22
13 31
13 4
13 1
13 37
13 7
13 25
13 36
13 14
13 32
13 6
13 8
13 10
13 13
13 3
13 24
13 41
13 29
13 28
13 34
13 38
13 12
13 27
13 40
13 17
13 21
13 18
13 15


In [ ]:

f2 = []


def join(item1, item2):
    if item1['class'] != item2['class']:
        return None
    condiset1 = set(item1['conditions'])
    condiset2 = set(item2['conditions'])

    if (condiset1 == condiset2):
        return None

    intersect = condiset1 & condiset2

    for item in intersect:
        if item1['conditions'][item] != item2['conditions'][item]:
            print("intersect not equal")
            return None

    condiset = condiset1 | condiset2
    new_item = {'conditions': {}, 'class': None}
    for item in condiset:
        if item in condiset1:
            new_item['conditions'][item] = item1['conditions'][item]
        else:
            new_item['conditions'][item] = item2['conditions'][item]
    new_item['class'] = item1['class']
    return new_item


for i in range(0, len(frequent_items_one)):
    for j in range(i+1, len(frequent_items_one)):
        new_item = join(frequent_items_one[i], frequent_items_one[j])
        if new_item != None:
            f2.append(new_item)

print(f2)


is_ruleitem(f2[0], data)

# print(is_ruleitem(f2[0], data))

[{'conditions': {0: 1, 1: 6}, 'class': 0}, {'conditions': {0: 1, 1: 4}, 'class': 0}, {'conditions': {0: 1, 1: 0}, 'class': 0}, {'conditions': {0: 1, 2: 0}, 'class': 0}, {'conditions': {0: 1, 2: 1}, 'class': 0}, {'conditions': {0: 1, 3: 9}, 'class': 0}, {'conditions': {0: 1, 3: 11}, 'class': 0}, {'conditions': {0: 1, 3: 15}, 'class': 0}, {'conditions': {0: 1, 4: 3}, 'class': 0}, {'conditions': {0: 1, 4: 2}, 'class': 0}, {'conditions': {0: 1, 4: 1}, 'class': 0}, {'conditions': {0: 1, 5: 4}, 'class': 0}, {'conditions': {0: 1, 5: 2}, 'class': 0}, {'conditions': {0: 1, 5: 0}, 'class': 0}, {'conditions': {0: 1, 6: 1}, 'class': 0}, {'conditions': {0: 1, 6: 4}, 'class': 0}, {'conditions': {0: 1, 6: 10}, 'class': 0}, {'conditions': {0: 1, 6: 8}, 'class': 0}, {'conditions': {0: 1, 6: 12}, 'class': 0}, {'conditions': {0: 1, 6: 3}, 'class': 0}, {'conditions': {0: 1, 6: 7}, 'class': 0}, {'conditions': {0: 1, 6: 0}, 'class': 0}, {'conditions': {0: 1, 7: 1}, 'class': 0}, {'conditions': {0: 1, 7: 0}, 

True